In [1]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense

class ActorNetwork(Model):
    def __init__(self, state_dim, action_dim):
        super(ActorNetwork, self).__init__()
        self._first_layer = Dense(256, activation='relu', input_shape= state_dim)
        self._second_layer = Dense(128, activation='relu')
        # Output 2 dimensional continous action [-1, 1]
        self._output_layer = Dense(action_dim[0], activation='tanh')
        self._state_dim = state_dim
        self._action_dim = action_dim

    # Return action tensorflow with shape (1, state_dim)
    def call(self, state):
        first_input = self._first_layer(state)
        second_input = self._second_layer(first_input)
        output_action = self._output_layer(second_input)
        return output_action

class CriticNetwork(Model):
    def __init__(self, state_dim, action_dim):
        super(CriticNetwork, self).__init__()
        self._first_state_layer = Dense(64, activation='relu', input_shape=(state_dim[0] + action_dim[0],))
        self._second_state_layer = Dense(32, activation='relu')
        # Output a single value which is the action-value
        self._output_layer = Dense(1, activation=None)
    
    # Output a value of dimension 1 shape (1, state_dim + action_dim)
    def call(self, state_action):
        first_state_input = self._first_state_layer(state_action)
        second_state_input = self._second_state_layer(first_state_input)
        output = self._output_layer(second_state_input)
        return output

In [2]:
import numpy as np

class ReplayMemory():
    def __init__(self, max_mem_size):
        self._max_mem_size = max_mem_size
        self._memory = []
        self._last_transition = ()
        self._mem_ctr = 0

    def store_transition(self, state, action, new_state, reward, terminal):
        transition = (state, action, new_state, reward, terminal)
        # If buffer is not full
        if len(self._memory) < self._max_mem_size:
            self._memory.append(transition)
        else:
            self._memory[self._mem_ctr] = transition
            # Circular buffer counter
            self._mem_ctr = (self._mem_ctr + 1) % self._max_mem_size

    def store_last_transition(self,state, action, new_state, reward, terminal):
        self._last_transition = (state, action, new_state, reward, terminal)

    def sample_transition(self, batch_size):
        last_transition = []
        if len(self._memory) < batch_size:
            last_transition.append(self._last_transition)
            return last_transition
        batch_indexes = np.random.choice(len(self._memory), batch_size, replace=False)
        samples = [self._memory[i] for i in batch_indexes]
        return samples


In [3]:
class OrnsteinUhlenbeckNoise:
    def __init__(self, action_dim, mu=0, theta=0.15, sigma=0.2):
        self.action_dim = action_dim[0]
        self.mu = mu
        self.theta = theta
        self.sigma = sigma
        self.state = np.ones(self.action_dim) * self.mu
        self.reset()

    def reset(self):
        self.state = np.ones(self.action_dim) * self.mu

    def sample(self):
        state = self.state
        dx = self.theta * (self.mu - state) + self.sigma * np.random.randn(self.action_dim)
        self.state = state + dx
        return self.state

In [5]:
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

class Agent():
    def __init__(self, state_dim, action_dim, gamma= 0.5, beta= 0.01,
                 capacity=100, mem_size= 100, learning_rate= 0.1, batch_size = 5):
        self._learning_rate = learning_rate
        self._capacity = capacity
        self._batch_size = batch_size
        self._gamma = gamma
        self._beta = beta
        self._mem_size = mem_size
        self._min_action = -1
        self._max_action = 1

        self._state_dim = state_dim
        self._action_dim = action_dim
        self._state_dim_net = state_dim[0] + action_dim[0]
        self._noise = OrnsteinUhlenbeckNoise(action_dim)

        self._replay_memory = ReplayMemory(self._mem_size)

        self._actor = ActorNetwork(state_dim, action_dim)
        self._target_actor = ActorNetwork(state_dim, action_dim)
        self._critic = CriticNetwork(state_dim, action_dim)
        self._target_critic = CriticNetwork(state_dim, action_dim)

        self._actor.compile(optimizer=Adam(self._learning_rate))
        self._critic.compile(optimizer=Adam(self._learning_rate))
        self._target_actor.compile(optimizer=Adam(self._learning_rate))
        self._target_critic.compile(optimizer=Adam(self._learning_rate))

    def get_action(self, state):
        # Convert state into tensor and reshape to feed into network
        numpy_state = np.array(state)
        reshaped_state = numpy_state.reshape(1, self._state_dim[0])
        tf_state = tf.convert_to_tensor(reshaped_state)
        action = self._actor(tf_state)
        clipped_action = tf.clip_by_value(action, self._min_action, self._max_action)

        return clipped_action.numpy()[0] + self._noise.sample() # add noise
 
    def store_transition(self, state, action, new_state, reward, terminal):
        self._replay_memory.store_transition(state, action, new_state, reward, terminal)
    
    def store_last_transition(self, state, action, new_state, reward, terminal):
        self._replay_memory.store_last_transition(state, action, new_state, reward, terminal)

    def update_target_actor_weights(self):
        actor_weights = self._actor.weights
        target_actor_weights = self._target_actor.weights
        updated_weights = []
        for i, weight in enumerate(actor_weights):
            updated_weights.append((weight*self._beta) + (1-self._beta)*target_actor_weights[i])
        self._target_actor.set_weights(updated_weights)

    def update_target_critic_weights(self):
        critic_weights = self._critic.weights
        target_critic_weights = self._target_critic.weights
        updated_weights = []
        for i, weight in enumerate(critic_weights):
            updated_weights.append((weight*self._beta) + (1-self._beta)*target_critic_weights[i])
        self._target_critic.set_weights(updated_weights)

    def gradient_descent_critic(self, state, new_state, reward):
        with tf.GradientTape() as tape:
            # Convert to tensors and reshape to feed to network
            action = self.get_action(state)
            state_action = np.concatenate((state, action))
            new_state_action = np.concatenate((new_state, action))
            tf_state_action = tf.convert_to_tensor(state_action.reshape(1, self._state_dim_net))
            tf_new_state_action = tf.convert_to_tensor(new_state_action.reshape(1, self._state_dim_net))

            # Feed to network and get target
            target_critic_value = self._target_critic(tf_new_state_action)
            critic_value = self._critic(tf_state_action)
            target = reward + self._gamma * target_critic_value
            critic_loss = tf.keras.losses.MSE(target, critic_value)

        # Perform gradient descent for critic with respect to critic weights
        critic_gradients = tape.gradient(critic_loss, self._critic.trainable_variables)
        self._critic.optimizer.apply_gradients(zip(critic_gradients, self._critic.trainable_variables))

    def gradient_ascent_actor(self, state):
        with tf.GradientTape() as tape:
            # Convert to tensors and reshape to feed to network
            action = self.get_action(state)
            state_action = np.concatenate((state, action))
            tf_state_action = tf.convert_to_tensor(state_action.reshape(1, self._state_dim_net))
            critic_value = self._critic(tf_state_action)
            actor_loss = - tf.reduce_mean(critic_value)
        
        # Perform gradient ascent for actor with respect to actor weights
        actor_gradients = tape.gradient(actor_loss, self._actor.trainable_variables)
        self._actor.optimizer.apply_gradients(zip(actor_gradients, self._actor.trainable_variables))

    def learn(self):
        samples = self._replay_memory.sample_transition(self._batch_size)
        for transition in samples:
            state, action, new_state, reward, terminal = transition

            self.gradient_ascent_actor(state)
            self.update_target_actor_weights()
            self.gradient_descent_critic(state, new_state, reward)
            self.update_target_critic_weights()


In [6]:
import gym

env = gym.make("LunarLander-v2", continuous= True, render_mode="human")
n_games = 1000
best_score = float('-inf')
episode_score_history = []
state_dim = env.observation_space.shape
action_dim = env.action_space.shape
agent = Agent(state_dim, action_dim)

for i in range(n_games):
    current_state = env.reset()[0]
    episode_score = 0
    terminal = False
    while not terminal:
        action = agent.get_action(current_state)
        new_state, reward, terminal, truncated, info = env.step(action)
        terminal = terminal or truncated
        agent.store_transition(current_state, action, new_state, reward, terminal)
        agent.store_last_transition(current_state, action, new_state, reward, terminal)
        current_state = new_state
        episode_score += reward
        agent.learn()

    episode_score_history.append(episode_score)
    avg_score = np.mean(episode_score_history[-100:])

    if avg_score > best_score:
        best_avg_score = avg_score
    print(f"Episode: {i}, score: {episode_score}, avg_score: {avg_score}, best_score: {max(episode_score_history)}")


c:\Users\nguye\AppData\Local\Programs\Python\Python39\lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


ValueError: No gradients provided for any variable: (['actor_network/dense/kernel:0', 'actor_network/dense/bias:0', 'actor_network/dense_1/kernel:0', 'actor_network/dense_1/bias:0', 'actor_network/dense_2/kernel:0', 'actor_network/dense_2/bias:0'],). Provided `grads_and_vars` is ((None, <tf.Variable 'actor_network/dense/kernel:0' shape=(8, 256) dtype=float32, numpy=
array([[-0.1270363 , -0.0531676 , -0.13230993, ..., -0.01138888,
        -0.02405429, -0.10280645],
       [-0.10561301,  0.03511579,  0.1497595 , ..., -0.05364694,
        -0.12176427,  0.09937266],
       [-0.02251211, -0.1274388 , -0.05422207, ...,  0.05141737,
        -0.09575436, -0.10565147],
       ...,
       [-0.09025693, -0.07140909, -0.14514272, ...,  0.08167745,
        -0.11591394,  0.11755019],
       [-0.11041129, -0.02023545, -0.12238231, ...,  0.12690616,
         0.01047863, -0.01740056],
       [ 0.11758155, -0.09517828, -0.10438947, ..., -0.0165707 ,
         0.11047366,  0.05663836]], dtype=float32)>), 